In [1]:
import plaidml.keras
plaidml.keras.install_backend()

In [2]:
# Подключаем библиотеки
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D

from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
# from keras.applications.resnet_v2 import ResNet152V2
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.densenet import DenseNet169
# from keras.applications.efficientnet import EfficientNetB6

In [33]:
import os

# Текущий каталог с проектом
current_dir = os.getcwd()

type(current_dir)

# Каталог с набором данных
data_dir = current_dir + "\\dogs-vs-cats\\train"

# Каталог с данными для обучения
train_dir = current_dir + "\\train"

# Каталог с данными для проверки
val_dir = current_dir + "\\val"

# Каталог с данными для тестирования
test_dir = current_dir + "\\test"

# Количество элементов данных в одном классе
nb_images = 12500

# Размеры изображения
img_width, img_height = 150, 150

# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)

# Размер мини-выборки
batch_size = 16

# Количество изображений для обучения
nb_train_samples = 17500

# Количество изображений для проверки
nb_validation_samples = 3750

# Количество изображений для тестирования
nb_test_samples = 3750

epochs = 1


In [4]:
def GetNetDict(weights, include_top, input_shape):
    list = {
        'VGG16' : VGG16(weights = weights, include_top= include_top, input_shape = input_shape),
        'Xception' : Xception(weights = weights, include_top= include_top, input_shape = input_shape),
        'VGG19' : VGG19(weights = weights, include_top= include_top, input_shape = input_shape),
        'ResNet50' : ResNet50(weights = weights, include_top= include_top, input_shape = input_shape),
        'InceptionV3' : InceptionV3(weights = weights, include_top= include_top, input_shape = input_shape),
        'InceptionResNetV2' : InceptionResNetV2(weights = weights, include_top= include_top, input_shape = input_shape),
        'MobileNet' : MobileNet(weights = weights, include_top= include_top, input_shape = input_shape),
        'MobileNetV2' : MobileNetV2(weights = weights, include_top= include_top, input_shape = input_shape),
        'DenseNet169' : DenseNet169(weights = weights, include_top= include_top, input_shape = input_shape)
    }
    return list

In [5]:
NetDict = GetNetDict(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"
C:\Users\artig\Anaconda3\envs\univer\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
C:\Users\artig\Anaconda3\envs\univer\lib\site-packages\keras_applications\mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '
C:\Users\artig\Anaconda3\envs\univer\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [6]:
# Создаем генератор изображений 
datagen = ImageDataGenerator(rescale=1. / 255)

# Генератор данных для обучения на основе изображений из каталога
train_generator = datagen.flow_from_directory(
 train_dir,
 target_size=(img_width, img_height),
 batch_size=batch_size,
 class_mode='binary')

# Генератор данных для проверки на основе изображений из каталога
val_generator = datagen.flow_from_directory(
 val_dir,
 target_size=(img_width, img_height),
 batch_size=batch_size,
 class_mode='binary')

# Генератор данных для тестирования на основе изображений из каталога
test_generator = datagen.flow_from_directory(
 test_dir,
 target_size=(img_width, img_height),
 batch_size=batch_size,
 class_mode='binary')

Found 17500 images belonging to 2 classes.
Found 3750 images belonging to 2 classes.
Found 3750 images belonging to 2 classes.


In [7]:
def FreezeWeghts(Net):
    Net.trainable = False

In [8]:
# "Замораживаем" веса предварительно обученной нейронной сети VGG16
for net in NetDict.items():
    FreezeWeghts(net[1])

In [9]:
def CreateCompositeModel(Net):
    # Создаем составную нейронную сеть на основе VGG16
    model = Sequential()

    # Добавляем в модель сеть VGG16 вместо слоя
    model.add(Net)

    # Слой преобразования данных из 2D представления в плоское
    model.add(Flatten())

    # Полносвязный слой для классификации
    model.add(Dense(256,  activation = 'relu'))

    # Слой регуляризации Dropout 
    model.add(Dropout(0.5))

    # Выходной полносвязный слой
    model.add(Dense(1, activation = 'sigmoid'))

    return model

In [10]:
CompositeModelDict = {}
for net in NetDict.items():
    CompositeModelDict.update({net[0] : CreateCompositeModel(net[1])})
    
for item in CompositeModelDict.items():
    item[1].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              P

In [11]:
# Компилируем составную нейронную сеть
def CompileNet(Net):
    Net.compile(loss='binary_crossentropy',
     optimizer='adam',
     metrics=['accuracy'])

In [12]:
for net in CompositeModelDict.items():
    CompileNet(net[1])

In [34]:
def FitNet(Net):
    Net.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)


In [16]:
for net in CompositeModelDict.items():
    print(net[0])
    FitNet(net[1])

VGG16
Epoch 1/2
1093/1093 [==============================] - 200s 183ms/step - loss: 0.3029 - acc: 0.8656 - val_loss: 0.2256 - val_acc: 0.9079
Epoch 2/2
1093/1093 [==============================] - 196s 179ms/step - loss: 0.2426 - acc: 0.8974 - val_loss: 0.2185 - val_acc: 0.9089
Xception
Epoch 1/2
1093/1093 [==============================] - 157s 144ms/step - loss: 1.1549 - acc: 0.8167 - val_loss: 0.1534 - val_acc: 0.9623
Epoch 2/2
1093/1093 [==============================] - 131s 120ms/step - loss: 0.2964 - acc: 0.8756 - val_loss: 0.1805 - val_acc: 0.9545
VGG19
Epoch 1/2
1093/1093 [==============================] - 258s 236ms/step - loss: 0.3644 - acc: 0.8402 - val_loss: 0.2713 - val_acc: 0.8809
Epoch 2/2
1093/1093 [==============================] - 248s 227ms/step - loss: 0.2888 - acc: 0.8775 - val_loss: 0.2641 - val_acc: 0.8907
ResNet50
Epoch 1/2
1093/1093 [==============================] - 148s 136ms/step - loss: 1.7703 - acc: 0.8797 - val_loss: 8.0504 - val_acc: 0.5005
Epoch 2/2
1

INFO:plaidml:Analyzing Ops: 788 of 2063 operations complete


 573/1093 [==============>...............] - ETA: 54s - loss: 0.5945 - acc: 0.7690

INFO:plaidml:Analyzing Ops: 1304 of 2063 operations complete


1093/1093 [==============================] - 145s 132ms/step - loss: 0.4959 - acc: 0.7967 - val_loss: 0.2327 - val_acc: 0.9447
Epoch 2/2
1092/1093 [============================>.] - ETA: 0s - loss: 0.3688 - acc: 0.8321

...

1092/1093 [============================>.] - ETA: 0s - loss: 7.9707 - acc: 0.4996

INFO:plaidml:Analyzing Ops: 802 of 2172 operations complete


1093/1093 [==============================] - 363s 332ms/step - loss: 7.9698 - acc: 0.4997 - val_loss: 7.9669 - val_acc: 0.5003
Epoch 2/2
1092/1093 [============================>.] - ETA: 0s - loss: 7.9669 - acc: 0.5003

INFO:plaidml:Analyzing Ops: 1531 of 2172 operations complete


1093/1093 [==============================] - 230s 211ms/step - loss: 7.9669 - acc: 0.5003 - val_loss: 7.9755 - val_acc: 0.4997


In [17]:
def SaveNet(NetName, Net):
    ## Сохраняем обученную нейронную сеть
    model_json = Net.to_json()
    json_file = open("dogs-vs-cats-model" + NetName + ".json", "w")
    json_file.write(model_json)
    json_file.close()
    Net.save_weights("dogs-vs-cats-model" + NetName + "gpu.h5")

In [18]:
for net in CompositeModelDict.items():
    SaveNet(net[0], net[1])

# Оставшиеся две сети

In [19]:
# Подключаем библиотеки
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D

from tensorflow.python.keras.applications.resnet_v2 import ResNet152V2
from tensorflow.python.keras.applications.efficientnet import EfficientNetB6

In [20]:
Resnet = ResNet152V2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
Efficient = EfficientNetB6(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

165240832/165234480 [==============================] - 20s 0us/step


In [21]:
Resnet.trainable = False
Efficient.trainable = False

In [29]:
Resnet_model = CreateCompositeModel(Resnet)
Efficient_model = CreateCompositeModel(Efficient)

In [30]:
CompileNet(Resnet_model)

In [31]:
CompileNet(Efficient_model)

In [35]:
FitNet(Resnet_model)
FitNet(Efficient_model)

1093/1093 [==============================] - 1154s 1s/step - loss: 1.1665 - accuracy: 0.4923 - val_loss: 0.6931 - val_accuracy: 0.4997


In [36]:
SaveNet('ResNet152V2', Resnet_model)
SaveNet('EfficientNetB6', Efficient_model)